<a href="https://colab.research.google.com/github/AhlemKaabi/yolov4-training-custom-dataset/blob/main/yolov4_training_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [ ]:
# this creates a symbolic link so that now the path 
!ln -s /content/gdrive/Shareddrives/Migration\ 2596 /kaabidrive

In [ ]:
# list the contents of /mydrive
!ls kaabidrive/

In [ ]:
#Navigate to /ahlemdrive/yolov4
%cd /kaabidrive/yolov4

/content/gdrive/MyDrive/yolov4


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 15490 (delta 10), reused 21 (delta 9), pack-reused 15460
Receiving objects: 100% (15490/15490), 14.08 MiB | 7.82 MiB/s, done.
Resolving deltas: 100% (10398/10398), done.
Checking out files: 100% (2057/2057), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/gdrive/MyDrive/yolov4/darknet


In [ ]:
# build darknet 
!make

In [ ]:
# Clean the data and cfg folders first except the labels folder in data whichis 
# is required

%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

In [ ]:
# Unzip the obj.zip dataset and its contents so that they are now in 
# /darknet/data/ folder 

!unzip /kaabidrive/yolov4/obj.zip -d data/

In [ ]:
# Copy the yolov4-custom.cfg file so that it is now in /darknet/cfg/ folder 

!cp /ahlemdrive/yolov4/yolov4-custom.cfg cfg

# verify if your custom file is in cfg folder
!ls cfg/

yolov4-custom.cfg


In [ ]:
# Copy the obj.names and obj.data files from your drive so that they are now
# in /darknet/data/ folder 

!cp /ahlemdrive/yolov4/obj.names data
!cp /ahlemdrive/yolov4/obj.data  data

# verify if the above files are in data folder
!ls data/

dataset_obj  labels  obj.data  obj.names


In [ ]:
# create  test.txt & train.txt that contains pathes to 10% & 90% respectivelly
# of the total images
# list the contents of data folder to check if the train.txt and test.txt 
# files have been created 
!ls data/

/content/gdrive/MyDrive/yolov4/darknet
labels	obj  obj.data  obj.names  old-dataset-obj  test.txt  train.txt


In [ ]:
# to start the training we will start with yolov4.conv.137 file weights then we 
# will save new weights untel the training is finished
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Training

In [ ]:
# train your custom detector!

# !./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

!./darknet detector train data/obj.data cfg/yolov4-custom.cfg /kaabidrive/yolov4/weights/4/yolov4-custom_last.weights -dont_show -map

# Test

In [ ]:
#set your custom cfg to test mode 
%cd /cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' yolov4-custom.cfg


In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test data/obj.data cfg/yolov4-custom.cfg /kaabidrive/yolov4/weights/5/yolov4-custom_final.weights /kaabidrive/yolov4_new/darknet/data/obj/39.jpg -thresh 0.3 -dont_show
imShow('predictions.jpg')

In [ ]:
# test on video
!./darknet detector demo data/obj.data cfg/yolov4-custom.cfg  /kaabidrive/yolov4/weights/5/yolov4-custom_final.weights -dont_show /kaabidrive/yolov4_new/Aircraft_turnaround.mp4 -i 0 -out_filename Aircraft_turnaround-with-1st-ds.avi